# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt_tab')
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import joblib


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

[nltk_data] Downloading package punkt to /Users/udtsv18/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/udtsv18/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/udtsv18/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/udtsv18/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/udtsv18/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///merged_data.db')
df = pd.read_sql_table('df', engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
def tokenize(text):
    # Normalize to lowercase
    text = text.lower()

    # Remove punctuation and non-alphanumeric characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    # Tokenize text
    tokens = word_tokenize(text)

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]

    return lemmatized_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # Tokenize and count word occurrences
    ('tfidf', TfidfTransformer()),  # Transform word counts to TF-IDF features
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Classify with RandomForest within MultiOutputClassifier
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x14ee9a480>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [11]:
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
for i, column in enumerate(Y.columns):
    print(f"Category: {column}")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))
    print("\n")

Category: related
              precision    recall  f1-score   support

           0       0.70      0.41      0.52      1266
           1       0.83      0.94      0.88      3938
           2       0.39      0.42      0.40        40

    accuracy                           0.81      5244
   macro avg       0.64      0.59      0.60      5244
weighted avg       0.80      0.81      0.79      5244



Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.48      0.61       895

    accuracy                           0.89      5244
   macro avg       0.86      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg      

/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
parameters = {
    'vect__max_df': [0.75],
    'vect__ngram_range': [(1, 1), (1, 2)],
    'clf__estimator__n_estimators': [50],
    'clf__estimator__min_samples_split': [2, 4]
}

X_sample, _, Y_sample, _ = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3, n_jobs=1)
cv.fit(X_sample, Y_sample)

print(f"Best parameters found: {cv.best_params_}")

Y_pred = cv.predict(X_test)


Fitting 2 folds for each of 4 candidates, totalling 8 fits


/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1);, score=0.253 total time=  39.9s


/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/2] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 1);, score=0.248 total time=  40.2s


...

[CV 1/2] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 2);, score=0.248 total time=  53.4s


/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/2] END clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 2);, score=0.242 total time=  53.0s


/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best parameters found: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 1)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
for i, column in enumerate(Y_test.columns):
    accuracy = accuracy_score(Y_test[column], Y_pred[:, i])
    precision = precision_score(Y_test[column], Y_pred[:, i], average='weighted')
    recall = recall_score(Y_test[column], Y_pred[:, i], average='weighted')
    print(f"Metrics for {column}:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(classification_report(Y_test[column], Y_pred[:, i]))

Metrics for related:
  Accuracy: 0.8063
  Precision: 0.7931
  Recall: 0.8063
              precision    recall  f1-score   support

           0       0.69      0.41      0.51      1266
           1       0.83      0.94      0.88      3938
           2       0.86      0.15      0.26        40

    accuracy                           0.81      5244
   macro avg       0.79      0.50      0.55      5244
weighted avg       0.79      0.81      0.79      5244

Metrics for request:
  Accuracy: 0.8947
  Precision: 0.8899
  Recall: 0.8947
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.48      0.61       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

Metrics for offer:
  Accuracy: 0.9950
  Precision: 0.9901
  Recall: 0.9950
              precision    recall  f1-score   support

        

/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/daniel_data_science/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
joblib.dump(cv, 'optimized_model.pkl')

['optimized_model.pkl']

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.